# Multithreaded computation with shared memory

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

## Two types of parallel dispatch

An easy job to parallelize is doing a simple thing to a lot of data. A common example here would be filtering a multi-channel signal.

<img src="images/vectorized_task.png" width="700pt">

This job can be split by splitting (muxing/demuxing) the dataset:

<img src="images/vectorized_task_para.png" width="700pt">

Splitting the large data array is a special case of the more general pattern of doing an arbitrary job for multiple input cases. An example for this case might be a grid-search to optimize hyperparameters in a machine learning design.

<img src="images/dispatched_jobs.png" width="700pt">

The general job dispatch pattern is covered in the JobRunner-demo notebook. This notebook will work through two examples of array splitting, and then look at the multiprocessing details behind parallelization.

### Contents

* Array splitting
  * Splitting existing methods
  * Applying splitting to new code
* Process start modes in Python multiprocessing
  + contextualized processesing, shared memory, etc
* Using shared memory
  + duality of numpy.ndarray and sharedctypes.Array
  + identical API for spawn & fork
* Logging contexts

## Array splitting

To demonstrate splitting a large array into smaller batches, we'll look at a good old, single-threaded Gaussian kernel filtering routine. This is effectively a finite impulse response (FIR) filter, where the complexity grows with the Gauss function width.

In [ ]:
from time import time
import numpy as np
from functools import wraps
from scipy.ndimage import gaussian_filter1d
from scipy.signal import lfilter

from ecogdata.parallel.mproc import parallel_context, make_stderr_logger
from ecogdata.parallel.array_split import split_at, split_output, split_optional_output

In [ ]:
# Use fork context if possible -- read below for details.

try:
    parallel_context.ctx = 'fork'
except:
    pass

Do a 100x Gaussian kernel FIRs with an order of 320 (+/- 4 sigma width, sigma=40)

In [ ]:
high_freq_signal = parallel_context.shared_copy(np.random.randn(100, 400000))

### Baseline single-threaded timing

In [ ]:
def timed_run(f, msg='runtime'):
    @wraps(f)
    def runner(*args, **kwargs):
        tic = time()
        r = f(*args, **kwargs)
        toc = time()
        print('{} {} sec'.format(msg, toc - tic))
        return r
    return runner

In [ ]:
sigma = 40
low_freq_signal = timed_run(gaussian_filter1d)(high_freq_signal, sigma, axis=1)

### Basic array splitting using the ``ecogdata.parallel.array_split.split_at`` "decorator"

A Python decorator is a function whose input and output are functions. It basically modifies the behavior of the original function. The function ``timed_run`` defined above is a type of decorator.

For array splitting, the decorator created within ``split_at`` drives the original function in subprocesses. Each subprocess acts on a separate batch of the full array(s). Decorators can be called with normal syntax, or using special syntax in combination with ``def`` 

```python
@split_at()
def normal_function(x):
    return 2 * x
```

This would create function that looks like ``normal_function``, but operates in subprocesses.

To parallelize an existing function, use normal syntax. The arguments say to split the array in the first argument position and to splice the output array in the first output position. Do this to see *all* possible configurations

```python
help(split_at)
```

In [ ]:
# doing this wrapping in two steps for clarity

# 1) parameterize the array splitting decorator
splitter = split_at(split_arg=(0,), splice_at=(0,))
# 2) transform the basic function using the decorator
gaussian_filter1d_para = splitter(gaussian_filter1d)

This may or may not show a speedup, depending on what process start method is active. More on that next.

In [ ]:
sigma = 40
low_freq_signal_2 = timed_run(gaussian_filter1d_para)(high_freq_signal, sigma, axis=1)

Test consistancy

In [ ]:
(low_freq_signal_2 == low_freq_signal).all()

### Smarter shared output splitting for simple input-output functions

The speedup is modest. Read the following sections for more details here. Briefly, the major *extra* costs of subprocess computation are

1. launching processes
1. sharing memory to processes
1. gathering results from processes

We only really have tools to improve (2) and (3). For getting memory to processes, we're already in good shape using shared memory:

```python
high_freq_signal = parallel_context.shared_copy(np.random.randn(100, 400000))
```

The parallel driver doesn't know anything about handling the output. So for the gathering step, the subprocesses actually have to "pickle" (serialize into a byte-stream) output arrays to be reconstituted in the main process. Better if we can put the output into shared memory that is already available to the main process.

It would be a two (or three) step process to modify the existing filter function

1. Define an outer function that gobbles the output into shared memory, rather than returning it

```python
def gaussian_filter1d_void(x, y, **kwargs):
    """Turn gaussian_filter1d into traditional C "void" behavior"""
    y[:] = gaussian_filter1d(x, **kwargs)
```

2. Then parallel-wrap the *void* function, with splitting on the input & output arguments

```python
gaussian_filter1d_para = split_at(split_arg=(0, 1))(gaussian_filter1d_void)
```

This function can be used with predefined shared output. Optional third step

3. Define convenience function to allocate new memory if needed

```python
def gaussian_filter1d_wrap(x, out=None, inplace=False, **kwargs):
    if out is None:
        if inplace:
            out = x
        else:
            out = parallel_context.shared_ndarray(x.shape, typecode=x.dtype.char)
    gaussian_filter1d_para(x, out, **kwargs)
    return out
```

### Special-purpose splitters for shared output

Two narrow purpose variations of ``split_at`` are defined for the fairly common scenario where the output is the same size as the input, and can be split into similiar subsets. These functions essentially do the three modifications above as part of the "decoration" process.

* ``split_output``: can be used for general functions that return an output the same size as the input
  + adds 'out' and 'inplace' keyword arguments to the wrapped method, like above
* ``split_optional_output``: used in the case where the original function has an option to put the results in a pre-defined array
  + also adds the 'inplace' option to the wrapped method

The first variation can (probably) be used in all such cases. However, it is probably most efficient to use the orignal function's output option if it's there.

In fact, gaussian_filter1d has an "output" option, so we'll use the second variation

In [ ]:
# doing this wrapping in two steps for clarity

splitter = split_optional_output(output_kwarg='output', split_arg=(0,))
gaussian_filter1d_para = splitter(gaussian_filter1d)

This can work three ways: 

1. the method creates new memory (default)
2. we define memory ahead of time (matching the single-thread behavior)
3. we do inplace (putting the output into the input)

In [ ]:
sigma = 40
low_freq_signal_2 = timed_run(gaussian_filter1d_para, msg='Default runtime')(high_freq_signal, sigma, axis=1)

In [ ]:
(low_freq_signal_2 == low_freq_signal).all()

In [ ]:
output = parallel_context.shared_ndarray(high_freq_signal.shape, typecode='d')
sigma = 40
low_freq_signal_2 = timed_run(gaussian_filter1d_para, 
                              msg='Predef memory runtime')(high_freq_signal, sigma, output=output, axis=1)

In [ ]:
(low_freq_signal_2 is output), (low_freq_signal_2 == low_freq_signal).all()

In [ ]:
sigma = 40
low_freq_signal_2 = timed_run(gaussian_filter1d_para,
                              msg='Inplace memory runtime')(high_freq_signal, sigma, axis=1, inplace=True)

In [ ]:
(low_freq_signal_2 is high_freq_signal), (low_freq_signal_2 == low_freq_signal).all()

## Applying splitters for novel code

This section applies some new tricks

* decorating a new function definition
* splitting general multidimensional arrays
* logging
* parallel state toggling

For spawned processes, parallelized code needs to be defined elsewhere and imported into the script. *The hack here is to write the same code definition to a temporary module. This is only for demonstration, and not normal practice.*

In [ ]:
from ecogdata.util import input_as_2d
from ecogdata.parallel.mproc import make_stderr_logger, parallel_controller

Decorate this function two ways. From inner to outer (order is important):

1. split with automatic output shared memory
2. reshape input/output arrays to be 2D (*this should be applied after splitting, since arrays are split on the first dimension*)

In [ ]:
if parallel_context.context_name == 'fork':
    @input_as_2d(in_arr=0, out_arr=0)
    @split_output(n_jobs=14)
    def convolve(x, kernel):
        y = np.empty_like(x)
        p = len(kernel)
        logger = parallel_context.get_logger()
        logger.info('{}'.format(x.shape))
        m, n = x.shape
        for i in range(p - 1, n):
            y[:, i] = x[:, i] * kernel[p - 1]
            y[:, i] = np.sum(x[:, i - (p - 1):i + 1] * kernel[::-1], axis=1)
        return y
else:
    code_def = """
import numpy as np
from ecogdata.util import input_as_2d
from ecogdata.parallel.array_split import split_output
from ecogdata.parallel.mproc import parallel_context


@input_as_2d(in_arr=0, out_arr=0)
@split_output(n_jobs=14)
def convolve(x, kernel):
    y = np.empty_like(x)
    p = len(kernel)
    logger = parallel_context.get_logger()
    logger.info('{}'.format(x.shape))
    m, n = x.shape
    for i in range(p - 1, n):
        y[:, i] = x[:, i] * kernel[p - 1]
        y[:, i] = np.sum(x[:, i - (p - 1):i + 1] * kernel[::-1], axis=1)
    return y
    """
    
    try:
        with open('temp_module.py', 'w') as f:
            f.write(code_def)
        from temp_module import convolve
    except:
        raise

In [ ]:
# general multidimensional ndarray to split

x = parallel_context.shared_copy(np.random.randn(10, 20, 20000))
kernel = np.ones(1500) / 15

The logging in the new function writes to the info channel. Under the error channel, nothing will show up.

In [ ]:
with make_stderr_logger('error'):
    y = timed_run(convolve)(x, kernel)

Under the info channel, we'll see the messages

In [ ]:
# there are no subprocess messages in spawn?
    
with make_stderr_logger('info'):
    y = convolve(x, kernel)

If parallel should be shut off, use the parallel controller. This can be set in a sticky way:

In [ ]:
parallel_controller.state = False
print('Parallel state is now:', bool(parallel_controller))
parallel_controller.state = True

Or better, in a way that only affect the context. Use it here to compare the parallel / serial timing.

In [ ]:
with parallel_controller(False):
    y = timed_run(convolve)(x, kernel)

In [ ]:
if parallel_context.context_name != 'fork':
    import os
    if os.path.exists('temp_module.py'):
        os.unlink('temp_module.py')

## Multiprocessing modes in Python

Parallel processing is supported in Python by way of a multiprocessing framework, where distinct processes are generated to run simultaneously. There are two primary modes that Python supports for creating subprocesses: "spawn" and "fork". (Another mode, "forkserver", is similar to spawn but not discussed here.) Mac and Linux support all subprocess types. Windows only support spawning. 

The tools discussed here are meant to operate consistently with any subprocess context. However, there are generally some limitations that will arise under spawning. These quirks (which I frankly don't understand too well) are related to saving and restoring the process state through "pickling". One major limitation with spawn affects interactive sessions (e.g. Notebooks). Generally, the code to parallelize should be defined elsewhere and *imported* to the session--that is all supported by pickling. Fork does not have that limitation, making interactive work a bit easier.

### Good practice for scripting

Parallelizing code that is *itself* multithreaded can lead to a resource race called "oversubscription," where each subprocess tries to launch into a routine that will consume multiple CPU cores or threads. This typically occurs in system libraries for linear algebra, fft, and certain other cases. You can set the runtime mode of certain libraries before linking to them, like this (not all variables apply to all systems, but it doesn't hurt to set them).

```python
import os
os.environ["OMP_NUM_THREADS"] = "1" # equivalent to shell: export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = "1" # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = "1" # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = "1" # export NUMEXPR_NUM_THREADS=1
```
    
## Run-time context management

While changing the default multiprocessing context is not usually, these use cases apply:

* choosing "fork" for interactive work, if available
* testing parallel code under multiple contexts

``ecogdata`` and similar libraries take advantage of a stateful parallel context that can be switched during runtime. The state object (``ecogdata.parallel.mproc.parallel_context``) has a name space that depends on what the present start mode is. In addition to the standard Python multiprocessing namespace, ``ecogdata.parallel`` adds a few conxtent-dependent shared memory tools as well (more below).

In [ ]:
parallel_context.ctx = 'fork'
print('Parallel mode:', parallel_context.context_name)
print('Mode objects:', parallel_context.Process, parallel_context.SharedmemManager)

The context can be changed either by reassignment, or in a contextualized block (using ``with`` syntax).

In [ ]:
with parallel_context.switch_context('spawn'):
    print('Parallel mode:', parallel_context.context_name)
    print('Mode objects:', parallel_context.Process, parallel_context.SharedmemManager)
# after leaving the context, parallel is switched to the previous state
print('Parallel mode:', parallel_context.context_name)

## Shared memory

A typical parallel pattern is to split a large amount of data into smaller batches. By default, data objects that are passed to a subprocess need to be "pickled", or serialized into a string of bytes, and then reconstructed in subprocess memory. This is usually true for forking and always true for spawning. Splitting a large array via serialization can be very time consuming, and duplicates memory consumption (*footnote for copy-on-write in forked processes).

Shared memory is a mechanism for providing a single data object that is accessible to the parent process and subprocesses. In particular, a numpy ndarray can have an underlying shared memory buffer, making it acessible by all processes. The module ``ecogdata.parallel.sharemem`` implements a ``SharedmemManager`` class that manages two faces of a common shared memory buffer:

* SharedmemManager.shm: ``multiprocessing.sharedctypes.Array``
* SharedmemManager.tonumpyarray: presents as a ``numpy.ndarray``


Creating shared memory is also handled through the manager via ``shared_ndarray`` 

```python
@classmethod
def shared_ndarray(cls, shape, typecode='d'):
    ...
```

and ``shared_copy``


```python
@classmethod
def shared_copy(cls, x):
    ...
```

### When to create shared memory

Complicating matters, the interaction of ndarrays and shared memory differs substantially in forked and spawned subprocesses. When spawning, all ``SharememManager`` objects point to actual shared memory, copying existing data if necessary. **With spawn, arrays that will be used with parallelized methods should be allocated as shared memory whenever possible.**

Under fork, there is more flexibility, and using shared memory requires some planning about whether ndarrays are created as shared memory or regular memory.  

**Subprocesses only do read-only access to arrays**

If a forked process is only *reading* from data arrays, it can use a memory pointer that looks like shared memory, even though it isn't. In this case, you don't need to create new shared memory. Shared memory managers can be created with *existing* ndarrays (takes advantage of the "copy-on-write" feature of forked process memory).

In the spawning context, creating a ``SharedmemManager`` for an existing array in plain memory will create a shared memory copy of that array. If that is a large array, you may want to allocate as shared memory to begin with.

**Subprocesses need read & write access to arrays**

Shared memory must be allocated. If "fake" shared memory is used, all write-backs to the array by the subprocess will be lost to the parent process.

**Subprocesses return large amounts of data**

Regardless of the read/write question, shared memory should be allocated to store the output data. This will avoid the time cost for serializing outputs back to the parent process.


### Memory management in basic Python 
In the future, the shared memory tools here may be replaced by the new SharedMemoryManager in Python. For the time being, this feature may be buggy (https://bugs.python.org/issue38119).